In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import time
import h5py
import os,sys
import numpy as np

In [3]:
DSPACK_HOME=os.environ.get("DSPACK_HOME")
if DSPACK_HOME not in sys.path:
    sys.path.append(DSPACK_HOME)

In [4]:
import scripts.fsystem
import scripts.statistics
reload(scripts.statistics)

<module 'scripts.statistics' from '/reg/data/ana03/scratch/zhensu/Develop/psvolume/v3.0/psvolume/scripts/statistics.pyc'>

In [5]:
workdir = "/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225/"
dnames = ["WT-1", "WT-2", "WT-3", "G150A-1", "G150A-2", "G150A-3", "G150T-1", "G150T-2", "G150T-3"]

In [12]:
def get_processing_results(pipeline="standard",scale_factor="profile",pca_step=True,dnames=None,workdir=None):
    data = np.zeros((6,9))
    file_tail = "_with_pca_map1.dsdata" if pca_step else "_map1.dsdata"
    
    ## Get CCFriedel
    for idx,dname in enumerate(dnames):
        dsdata_file = "%s/data/%s/%s_clean_data_scale_%s"%(workdir,dname,pipeline,scale_factor)+file_tail
        map_file = scripts.fsystem.H5manager.reader(dsdata_file,"merge_volume")
        pdb_file = scripts.fsystem.H5manager.reader(dsdata_file,"pdb_refmac5")
        data[1, idx] = scripts.statistics.calculate_ccfriedel_from_map(map_file,pdb_file)
        if dname=="WT-1":
            print dsdata_file
        
    ## Get CCLaue
    for idx,dname in enumerate(dnames):
        dsdata_file = "%s/data/%s/%s_clean_data_scale_%s"%(workdir,dname,pipeline,scale_factor)+file_tail
        map_file = scripts.fsystem.H5manager.reader(dsdata_file,"merge_volume")
        pdb_file = scripts.fsystem.H5manager.reader(dsdata_file,"pdb_refmac5")
        data[2, idx] = scripts.statistics.calculate_cclaue_from_map(map_file,pdb_file)
        if dname=="WT-1":
            print dsdata_file
        
    ## Get CC1/2
    for idx,dname in enumerate(dnames):
        dsdata_file = "%s/data/%s/%s_clean_data_scale_%s"%(workdir,dname,pipeline,scale_factor)+file_tail
        phenix_file = scripts.fsystem.H5manager.reader(dsdata_file,"phenix_merge_stats")
        data[3, idx] = scripts.fsystem.H5manager.reader(phenix_file,"phenix_cc12")
        
        phenix_dat = os.path.join(os.path.dirname(phenix_file),"phenix.merging_statistics.out")
        with open(phenix_dat) as f:
            content = f.readlines()
        data[0, idx] = float(content[-8].split()[5])
        if dname=="WT-1":
            print dsdata_file
        
    ## Get CCRep
    marker = np.triu(np.ones((9,9)).astype(int), 1)
    marker[0:6,6:9] = 0
    marker[0:3,3:6] = 0
    
    map_file_tail = "_with_pca.out" if pca_step else ".out"
    path_cc_map = "%s/data/CC-MAP/ccmap_%s_clean_data_scale_%s"%(workdir,pipeline,scale_factor)+map_file_tail
    ccmap = scripts.fsystem.H5manager.reader(path_cc_map,"cc")
    ccmap *= marker
    data[4] = np.sum(np.maximum(ccmap,ccmap.T),axis=0) / np.sum(np.maximum(marker,marker.T),axis=0)
    print path_cc_map
    
    ## Get CC Cross
    marker = np.zeros((9,9))
    marker[0:3,3:6] = 1
    ccmap = scripts.fsystem.H5manager.reader(path_cc_map,"cc")
    ccmap *= marker
    tmp = np.sum(np.maximum(marker,marker.T),axis=0)
    tmp[tmp==0]=1e9
    data[5] = np.sum(np.maximum(ccmap,ccmap.T),axis=0) / tmp
    ## 
    return data

In [13]:
def print_data(data,ref):
    data_name = ["Complete","CC_Friedel","CC_Laue","CC_1/2","CC_Rep","CC_Cross"]
    for i in range(len(data)):
        x = []
        for j in range(len(data.T)):
            value = "{:.2f}".format(data[i,j] - ref[i,j])
            if data[i,j] - ref[i,j] > 0:
                value = "+"+value
            x.append(value)
        print str(data_name[i]).ljust(12), " ".join([_.ljust(8) for _ in x])

In [14]:
data_name = ["Complete","CC_Friedel","CC_Laue","CC_1/2","CC_Rep","CC_Cross"]

## A. Standard Pipeline

In [15]:
data = get_processing_results(pipeline="standard",scale_factor="profile",pca_step=True,dnames=dnames,workdir=workdir)
ref = data.copy()
## Print the data
for idx,x in enumerate(data):
    print str(data_name[idx]).ljust(12)," ".join([str(round(i,2)).ljust(8) for i in x])

/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/standard_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/standard_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/standard_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/CC-MAP/ccmap_standard_clean_data_scale_profile_with_pca.out
Complete     98.36    100.0    99.3     100.0    98.8     100.0    99.84    100.0    98.97   
CC_Friedel   0.93     0.91     0.91     0.93     0.91     0.91     0.91     0.92     0.94    
CC_Laue      0.9      0.87     0.87     0.88     0.86     0.85     0.86     0.88     0.91    
CC_1/2       0.85     0.78     0.81     0.81     0.76     0.77     0.82     0.84     0.89    
CC_Rep       0.86     0.84     0.85     0.82     0.81     0.82     0.88     0.87     0.89    
CC_Cross     0.83     0.84     0.85     0.84  